In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba

import re

In [2]:
# 训练集
train_df = pd.read_csv('training_set.txt', sep="\t", header=None, names=["label", "sentence"])
val_df = pd.read_csv('validation_set.txt', sep="\t", header=None, names=["label", "sentence"])
train = pd.concat([train_df,val_df],axis=0)
train.head()

# 打乱训练集的顺序
train = train.sample(frac=1, random_state=1)
train.reset_index(inplace=True, drop=True)

In [3]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest,Ytrain,Ytest = train_test_split(train["sentence"],train["label"],test_size=0.3)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
# 统计词频信息
count_vector = CountVectorizer(max_df=0.8, min_df=3)
vecot_matrix= count_vector.fit_transform(Xtrain)
# vecot_matrix.toarray()
# count_vector.vocabulary_

from sklearn.feature_extraction.text import TfidfTransformer
transformer=TfidfTransformer()
tfidf = transformer.fit_transform(vecot_matrix)

In [5]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# X_train_vect = vect.fit_transform(X_train)
nb.fit( tfidf, Ytrain)
train_score = nb.score(tfidf, Ytrain)
print(train_score)

# result 0.8912458097488597

0.8912458097488597


In [6]:
test_tfidf = transformer.fit_transform(count_vector.transform(Xtest))

In [8]:
# 查看测试集上的效果
test_score = nb.score(test_tfidf,Ytest)
test_score
# 0.8379279394794205

0.83690216694448

In [38]:
df = pd.read_csv(r"film8w5.csv",encoding = 'utf-8',lineterminator="\n")
df.head()

,film,user,star,time,content
0,007之黑日危机,yf747126,还行,2006-05-07,一部还算成功的商业片，我永远都不明白为什么女人会喜欢数字先生
1,007之黑日危机,托尼·王大拿,较差,2017-04-20,虽然苏菲·玛索人美戏好、丹妮丝·理查兹人美胸大，但咋又回到反派间歇性脑抽的老邦德路线了：直升...
2,007之黑日危机,南條某葉,推荐,2013-01-30,007-19，我以为我没看过，下下来一看原来是看过的= =b，顺便：苏菲玛索好美！
3,007之黑日危机,乐事,还行,2009-08-10,布卢斯皮尔斯南的007系列，还可以吧，不同的就是这部中有苏菲玛索饰演的，反面邦女郎。很美！
4,007之黑日危机,躺着也中枪,还行,2012-10-27,可惜了苏菲玛索这妹子，居然演的反派\r


In [39]:
# 分词 + 去除停用词
def deleteStopWord(sentence):
    with open(r"stop_words.txt",'r',encoding='utf-8') as f:
        stop_word = f.read()
    s = ''
    sentence = ' '.join(jieba.cut(sentence))
    for i in sentence.split(" "):
        if i not in stop_word:
            s += i + " "
    return s

df['content'] = df['content'].map(lambda c : deleteStopWord(c))

df.head()

In [41]:
df_tfidf = transformer.fit_transform(count_vector.transform(df["content"]))

In [42]:
predict = nb.predict(df_tfidf)

In [43]:
df["nb_predict"] = pd.DataFrame(predict)

In [44]:
df["nb_predict"].value_counts()

0    51169
1    33677
Name: nb_predict, dtype: int64

### 根据时间统计正负评的数量

In [157]:
# 改变时间格式
def changeTime(c):
    c = c.split("-")[0]
    return c
# changeTime("2006-05-07")
df['time'] = df['time'].map(lambda c : changeTime(c) )

In [180]:
positive = df.groupby(by=["time"])['nb_predict'].sum()
positive = pd.DataFrame(positive)

# 计算每年负评论的数量
positive.reset_index(inplace=True)


In [221]:
positive

,time,nb_predict
0,2005,68
1,2006,451
2,2007,841
3,2008,2365
4,2009,2578
5,2010,4193
6,2011,5439
7,2012,4754
8,2013,2877
9,2014,1637


In [252]:
time_count = pd.DataFrame(df['time'].value_counts())
time_count.reset_index(inplace=True)
time_count.columns = ['time','count']
time_count = time_count.sort_values(by='time' ,ascending=True)
time_count.reset_index(inplace=True,drop=True)
time_count

,time,count
0,2005,183
1,2006,1100
2,2007,2182
3,2008,5789
4,2009,6636
5,2010,10842
6,2011,13491
7,2012,11827
8,2013,6882
9,2014,3937


In [255]:
negative = time_count.copy()
negative['count'] = time_count['count'] - positive['nb_predict']

In [256]:
negative['count'].sum()

51668

In [260]:
# print(positive['time'] == negative['time'])
print(list(positive['time']))
print(list(negative['count']))
print(list(positive['nb_predict']))

['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
[115, 649, 1341, 3424, 4058, 6649, 8052, 7073, 4005, 2300, 1186, 1826, 3865, 2907, 2398, 1820]
[68, 451, 841, 2365, 2578, 4193, 5439, 4754, 2877, 1637, 751, 1069, 2115, 1463, 1369, 1208]


In [257]:
len(df['nb_predict']) - df['nb_predict'].sum()

51668

In [118]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [120]:
clf = DecisionTreeClassifier(random_state=1)
clf = clf.fit(tfidf,Ytrain)

score_c = clf.score(test_tfidf,Ytest)

In [121]:
score_c

0.6757276573919734

In [122]:
rfc = RandomForestClassifier(random_state=1)
rfc = rfc.fit(tfidf,Ytrain)
score_r = rfc.score(test_tfidf,Ytest)

In [123]:
score_r

0.7897166303372227

### 统计每部电影正向情感、负向情感每个星级的分数

In [141]:
df1 = df.copy()
df = df.query("star != '暂未评价'")

In [145]:
positive_df = df[df['nb_predict'].isin(['1'])]
negative_df = df[df['nb_predict'].isin(['0'])]

In [146]:
def getRadar(dataframe):
    radar_df = pd.DataFrame(dataframe['star'].groupby(dataframe['film']).value_counts())
    radar_df = radar_df.unstack()
    return radar_df
positive_rader = getRadar(positive_df)
negative_radar = getRadar(negative_df)